Play MimbleWimble with klefki
=========================



In [1]:
from klefki.types.algebra.concrete import (
    EllipticCurveGroupSecp256k1 as ECG,
    FiniteFieldSecp256k1 as F,
    FiniteFieldCyclicSecp256k1 as CF
)
import random
import hashlib
from klefki.utils import to_sha256int
N = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
def random_cf() -> CF: return CF(random.randint(1, N) % F.P)
G = ECG.G

In [2]:
s = bytes.fromhex("0479be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8")
x = int(hashlib.sha256(s).hexdigest(),16)

In [3]:
H = ECG.lift_x(CF(0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F))

## Transacting with MimbleWimble

The validation of MimbleWimble transactions relies on two basic properties:


* Verification of zero sums.

The sum of outputs minus inputs always equals zero, proving that the transaction did not create new funds, without revealing the actual amounts.
    
    
* Possession of private keys. 

Like with most other cryptocurrencies, ownership of transaction outputs is guaranteed by the possession of ECC private keys. However, the proof that an entity owns those private keys is not achieved by directly signing the transaction.


### Balance

If v is the value of a transaction input or output and H a point on the elliptic curve C, we can simply embed $H^v$ instead of v in a transaction. This works because using the ECC operations, we can still validate that the sum of the outputs of a transaction equals the sum of inputs:



In [4]:
v1, v2 = random_cf(), random_cf()

In [5]:
v3 = v1 + v2
H@v3 == (H@v1) + (H@v2)

False

In [6]:
H@v3 == H@(v1+v2)

True

An input or output value in a transaction can then be expressed as:

$$
rG + vH
$$

Where:

$r$ is a private key used as a blinding factor.

$v$ is the value of an input or output and H is another point on the elliptic curve C

As an example, let's assume we want to build a transaction with two inputs and one output. We have (ignoring fees):

* $vi1$ and $vi2$ as input values
* $vo3$ as output value.

Such that:

$$
vi1 + vi2 = vo3
$$

Generating a private key as a blinding factor for each input value and replacing each value with their respective Pedersen Commitments in the previous equation, we obtain:

$$
(ri1G + vi1H) + (ri2G + vi2H) = (ro3G + vo3H)
$$

Which as a consequence requires that:

$$
ri1 + ri2 = ro3
$$

In [ ]:
v1 = CF(3)
v2 = CF(97)
v3 = CF(100)

assert v1 + v2 == v3

In [ ]:
r1, r2 = random_cf(), random_cf()

r3 = r1 + r2

In [ ]:
G@r1 + H@v1 + G@r2 + H@v2 == G@r3 + H@v3

In [ ]:
    # for y^2 = x^3 + 7
    
G.value[1]**2 == G.value[0]**3 + F(7)

In [ ]:
import math
math.sqrt((G.value[0]**3 + F(7)).value)

In [ ]:
G.value[1]

### Ref:

[1] Intro (Grin) https://github.com/mimblewimble/grin/blob/master/doc/intro.md